<a href="https://colab.research.google.com/github/allkon14/Visual-Counting-Ability-of-Tiny-VLMs/blob/main/task_11_Visual_Counting_Ability_of_Tiny_VLMs_Konopleva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparations


## Installs and imports nessesary libraries


In [ ]:
!pip install -q accelerate peft bitsandbytes tensorboard torch transformers datasets huggingface_hub trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 52.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


After further training, I will save the model to Hugging Face, so you need to log in

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import re
import csv
import pandas as pd
import random
from tqdm import tqdm
import numpy as np
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq, AutoTokenizer, BitsAndBytesConfig, Idefics3ForConditionalGeneration
from transformers.image_utils import load_image
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from google.colab import files
from sklearn.metrics import accuracy_score


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
torch.cuda.is_available()

True

## Load and initialize model and processor

In [ ]:
# Initialize processor and model
model_id = "HuggingFaceTB/SmolVLM-256M-Instruct"
processor = AutoProcessor.from_pretrained(model_id)
model_zs = AutoModelForVision2Seq.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    _attn_implementation="eager",
).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


In this notebook I will not do full fine-tuning, but used the QLoRA method, which loads the adapter for the quantized version of the model, saving space.



In [ ]:
USE_LORA = False
USE_QLORA = True
SMOL = True

processor = AutoProcessor.from_pretrained(
    model_id
)

if USE_QLORA or USE_LORA:
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
        use_dora=False if USE_QLORA else True,
        init_lora_weights="gaussian"
    )
    lora_config.inference_mode = False
    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )

    model_stf = AutoModelForVision2Seq.from_pretrained(
        model_id,
        quantization_config=bnb_config if USE_QLORA else None,
        _attn_implementation="eager",
        device_map="auto"
    )
    model_stf.add_adapter(lora_config)
    model_stf.enable_adapters()
    model_stf = prepare_model_for_kbit_training(model_stf)
    model_stf = get_peft_model(model_stf, lora_config)
    print(model_stf.get_nb_trainable_parameters())
else:
    model_stf = AutoModelForVision2Seq.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        _attn_implementation="eager",
    ).to(DEVICE)

    for param in model.model.vision_model.parameters():
        param.requires_grad = False

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


(2884608, 259369536)


## Load dataset and split it

In [ ]:
# load dataset
ds = load_dataset("leonardPKU/clevr_cogen_a_train")

README.md:   0%|          | 0.00/364 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

train-00000-of-00027.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

train-00001-of-00027.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

...

train-00024-of-00027.parquet:   0%|          | 0.00/491M [00:00<?, ?B/s]

train-00025-of-00027.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

train-00026-of-00027.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/70000 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
df = ds['train']
df[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=480x320>,
 'problem': 'How many items are there in the image?',
 'solution': '<answer> 3 </answer>'}

Splitting the data set (the size of the split was chosen experimentally so that Colab resources were sufficient and that training did not take too long). The data is mixed up.

In [ ]:
train_test_split_df = ds['train'].train_test_split(test_size=0.95, shuffle=True)
train_val_split_df = train_test_split_df['train'].train_test_split(test_size=0.1, shuffle=True)
test_split_ds = train_test_split_df['test'].train_test_split(test_size=0.015, shuffle=True)

In [ ]:
train_ds, val_ds, test_ds = train_test_split_df['train'], train_val_split_df['test'], test_split_ds['test']
print(f"Train: {len(train_ds)} | Validation: {len(val_ds)} | Test: {len(test_ds)} | Total: {len(train_ds) + len(val_ds) + len(test_ds)}")

Train: 3500 | Validation: 350 | Test: 998 | Total: 4848


To test the model without training and after fine-tuning, the same test dataset was used (to assess the basic quality of the model and understand how effective the fine-tuning was)

Example of a message that is sent to the model input

In [ ]:
messages = [
{
    "role": "user",
    "content": [
        {"type": "image"},
        {"type": "text", "text": 'How many items are there in the image?'}
    ]
},
]

## Additional functions

In [ ]:
def extract_numbers(text):
    """Extracting numbers from a string"""
    num = re.search(r'\d+', text).group() if re.search(r'\d+', text) else None
    return num

In [ ]:
def replace_nums(s):
    """ Replacing the alphabetic spelling of numbers with their numeric version

        (required for processing the response from the model, as it may not use numbers in its response)"""

    d = {'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9', 'ten': '10'}
    for k, v in d.items():
        if k in s:
            s = s.replace(k, v)
    return s

In [ ]:
def get_model_answer(df, message, model, index=None, x=None, transform=False):
        """
        Getting a response from a model based on a df dataset instance,
        which can be  taken by index or passed as an argument,
        when requesting a message.

        Return: Model prediction and correct answer

        df - data set
        message - request to the model
        model - the model being tested
        index - index of the object in the data set
        x - instance of the dataset
        transform-parameter that indicates whether numeric values should be extracted from predictions and correct answers

        """
    if index is not None:
        image = df[index]['image']
        ans = df[index]['solution']

    elif x is not None:
        image = x['image']
        ans = x['solution']

    # Prepare inputs
    prompt = processor.apply_chat_template(message, add_generation_prompt=True)
    inputs = processor(text=prompt, images=[image], return_tensors="pt")
    inputs = inputs.to(DEVICE)

    # Generate outputs
    generated_ids = model.generate(**inputs)
    generated_texts = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
    )

    if transform:
        return extract_numbers(replace_nums(generated_texts[0].split('Assistant: ')[-1])), extract_numbers(ans)
    else:
        return generated_texts[0], ans

In [ ]:
def check_data(model, file_name, data, mode='sample', save_lists=True):
   """
    Sending model requests to get predictions (requests are sent one by one to the model, so the output is one prediction per request)

    model - the model being tested
    file_name - the name of the file (for saving)
    data - the dataset
    mode = ['sample', 'index'] - using the elements themselves ('sample') or accessing them by index ('index')
    save_lists - whether to save predictions and correct answers in lists
    """
    if save_lists:
        predictions = np.array([])
        correct_answers = np.array([])

    with open(file_name, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)

        if mode=='index':
            writer.writerow(["Object_Index", "Model Prediction", "Correct Answer"])
            for i in tqdm(sample):
                model_prediction, correct_answer = get_model_answer(data, messages, model, index=i, transform=True)
                writer.writerow([i, model_prediction, correct_answer])
                if save_lists:
                    predictions = np.append(predictions, int(model_prediction))
                    correct_answers = np.append(correct_answers, int(correct_answer))

        if mode=='sample':
            writer.writerow(["Model Prediction", "Correct Answer"])
            for x in tqdm(data):
                model_prediction, correct_answer = get_model_answer(data, messages, model, x=x, transform=True)
                writer.writerow([model_prediction, correct_answer])
                if save_lists:
                    predictions = np.append(predictions, int(model_prediction))
                    correct_answers = np.append(correct_answers, int(correct_answer))

    files.download(file_name)
    print(f'File {file_name} has been downlowded')

    if save_lists:
        return [predictions, correct_answers]


In [ ]:
# A function that uses a template to combine questions and answers
# so that the model can learn how to answer questions.
# After that, we pass the formatted data and images to the processor, which processes them.

image_token_id = processor.tokenizer.additional_special_tokens_ids[
            processor.tokenizer.additional_special_tokens.index("<image>")]

def collate_fn(examples):
    """Collate function to format multiple examples into a batch

        examples - multiple sample dataset (batch)
    """
    texts = []
    images = []
    for example in examples:
        image = example["image"]
        if image.mode != 'RGB':
            image = image.convert('RGB')
        question = example["problem"]
        answer = extract_numbers(example["solution"])
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Answer briefly."},
                    {"type": "image"},
                    {"type": "text", "text": question}
                ]
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": answer}
                ]
            }
        ]
        text = processor.apply_chat_template(messages, add_generation_prompt=False)
        texts.append(text.strip())
        images.append([image])

    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    batch["labels"] = labels

    return batch

# Zero-Shot Setup

Data was submitted one instance of the test set according to the input data that the model expected. All results were saved to a csv file.

In [ ]:
file_name = 'zero-shot_test_ds_model_predictions.csv'
predictions, correct_answers = check_data(model_zs, file_name, test_ds, mode='sample', save_lists=True)

100%|██████████| 998/998 [11:45<00:00,  1.42it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File zero-shot_test_ds_model_predictions.csv has been downlowded


In [ ]:
pd_df = pd.read_csv(file_name)

pd_df.head(10)

,Model Prediction,Correct Answer
0,3,3
1,3,3
2,4,4
3,6,6
4,9,7
5,10,10
6,9,7
7,9,9
8,4,5
9,9,6


In [ ]:
pd_df.shape

(998, 2)

Calculating accuracy ($\frac{\text{number of correct predictions}}{\text{number of all predictions}}$)

Using a scikit-learn library method

In [ ]:
accuracy_score_zero = accuracy_score(predictions, correct_answers)
accuracy_score_zero

0.6492985971943888

By yourself

In [ ]:
pd_df[pd_df['Correct Answer'] == pd_df['Model Prediction']].shape[0]

648

In [ ]:
pd_df[pd_df['Correct Answer'] == pd_df['Model Prediction']].shape[0] / pd_df.shape[0]

0.6492985971943888

Check if the model was wrong exactly one value from the true value

In [ ]:
pd_df[abs(pd_df['Correct Answer'] - pd_df['Model Prediction']) <= 1].shape[0] / pd_df.shape[0]

0.8517034068136272

**Conclusion**: without additional training, the model on a random test sample of 998 objects gave the correct answer in 64.9% of cases (correctly predicted the answer to 648 objects out of 998).

Additionally, the model was tested on a slightly larger test dataset (1750 objects) (one of the first attempts). In that case, without additional training, the model on a random test sample of 1750 instances gave the correct answer in 48.4% of cases (correctly predicted the answer on 847 objects out of 1750). On the remaining objects, the model was wrong by one.

# Supervised Fine-Tuning (SFT) Setup

For additional training, we used `Trainer`, an API for full-fledged training in PyTorch, and `TrainingArguments`, which offers a wide range of parameters for configuring model training.

Gradient accumulation technology was used to create the effect of a larger package. In addition, using gradient control points saved memory for intermediate activations.

In [ ]:
model_name = model_id.split("/")[-1]

training_args = TrainingArguments(
    num_train_epochs=1,  # number of epochs
    per_device_train_batch_size=2,  # batch size during training
    gradient_accumulation_steps=4,  # number of steps before updating model weights (gradients will be updated every 4 steps)
    warmup_steps=20,  # number of warmup steps, where the learning rate gradually increases to its main value (helps avoid gradient spikes at the beginning of training)
    learning_rate=1e-4,  # learning rate
    weight_decay=0.01,  # L2 regularization coefficient, prevents overfitting
    logging_steps=20,  # model will log every 20 steps
    save_strategy="steps",  # strategy for saving the model (in this case, the model will be saved after a certain number of steps)
    save_steps=50,  # model will be saved every 50 steps
    save_total_limit=1,  # number of models to keep during training
    optim="paged_adamw_8bit",  # uses the 8-bit AdamW optimizer, which reduces memory usage and speeds up training
    bf16=True,  # use of bfloat16
    output_dir=f"./{model_name}-stf",  # directory to save the model
    hub_model_id=f"{model_name}-stf",  # model ID for uploading to Hugging Face Hub
    report_to="tensorboard",  # defines where to send logs
    remove_unused_columns=False,  # allows not to remove unused columns from the dataset
    gradient_checkpointing=True  # allows saving intermediate gradient computations and reduces memory usage by recalculating them during the backward pass
)


The training process on the A100 GPU required about 11 GB of video memory (due to the small amount of training data)

In [ ]:
trainer = Trainer(
    model=model_stf,    # model to be fine-tuned
    args=training_args, # training parameters
    data_collator=collate_fn,   # data preprocessing function
    train_dataset=train_ds, # training dataset
    eval_dataset=val_ds,    # validation dataset
)

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
20,2.356300
40,0.556600
60,0.259400
80,0.211600
100,0.171100
120,0.125800
140,0.032200
160,0.003200
180,0.002300
200,0.003400


TrainOutput(global_step=437, training_loss=0.17133475462234743, metrics={'train_runtime': 3666.9321, 'train_samples_per_second': 0.954, 'train_steps_per_second': 0.119, 'total_flos': 4274444869187328.0, 'train_loss': 0.17133475462234743, 'epoch': 0.9988571428571429})

In [ ]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

events.out.tfevents.1742024234.710714e6b6a1.837.0:   0%|          | 0.00/13.6k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/allko14/SmolVLM-256M-Instruct-stf/commit/69699e16d45ad88ee7f4d500526b19b0ee8cadf1', commit_message='End of training', commit_description='', oid='69699e16d45ad88ee7f4d500526b19b0ee8cadf1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/allko14/SmolVLM-256M-Instruct-stf', endpoint='https://huggingface.co', repo_type='model', repo_id='allko14/SmolVLM-256M-Instruct-stf'), pr_revision=None, pr_num=None)

Fine-tuning the model took place during one epoch on a small training dataset (3500 objects in total)

## Testing model after fine-tuning

Two objects were fed to the model to get the prediction. Objects were converted using the additional function `collate_fn`, which returned data to be passed to the model input in the format that it needed. After receiving the prediction from the model, the numbers were extracted from the response and stored in a list.

In [ ]:
ft_model_predictions = []

for i in tqdm(range(0, len(test_ds), 2)):
    examples = test_ds.select(range(i, i+2))
    batch = collate_fn(examples)
    inputs = batch.to(DEVICE)

    generated_ids2 = model_stf.generate(**inputs)
    generated_texts2 = processor.batch_decode(
        generated_ids2,
        skip_special_tokens=True,
    )

    ftm_pred = [(extract_numbers(gts.split('\nAnswer: ')[0].split('\nAssistant: ')[1])) for gts in generated_texts2]
    ft_model_predictions.extend(ftm_pred)


  0%|          | 0/499 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
  0%|          | 2/499 [00:05<22:19,  2.70s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  2%|▏         | 8/499 [00:21<21:39,  2.65s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  2%|▏  

In [ ]:
# storing true labels of the test dataset
true_answers = list(map(extract_numbers, test_ds['solution']))

In [ ]:
# calculating accuracy
accuracy_ftm = accuracy_score(ft_model_predictions, true_answers)
accuracy_ftm

1.0

In [ ]:
# saving model responses after fine tuning and true responses
with open("model_after_fine-tuning_pred_test_ds.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    writer.writerow(["Fine-tuning Model Prediction", "Correct Answer"])
    for ft, ans in zip(ft_model_predictions, true_answers):
        writer.writerow([ft, ans])

files.download('model_after_fine-tuning_pred_test_ds.csv')
print(f'File model_after_fine-tuning_pred_test_ds.csv has been downlowded')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File model_after_fine-tuning_pred_test_ds.csv has been downlowded


As you can see, after fine-tuning the model shows 100% accuracy. It is important to note that the input data was in the version that the model expects.

Below is an example of testing a model when the input data was different from what the model expected.

In [ ]:
file_name = 'ft_model_test_ds_model_predictions.csv'
predictions_ft, correct_answers_ft = check_data(model_stf, file_name, test_ds, mode='sample', save_lists=True)

100%|██████████| 998/998 [18:38<00:00,  1.12s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File ft_model_test_ds_model_predictions.csv has been downlowded


In [ ]:
# calculating accuracy
accuracy_ft = accuracy_score(predictions_ft, correct_answers_ft)
accuracy_ft

0.34168336673346694

## Load from pretrained

Loading the model and adapter for the model that was fine-tuned

In [ ]:
model_id = "HuggingFaceTB/SmolVLM-256M-Instruct"
processor = AutoProcessor.from_pretrained(model_id)
model_hf = AutoModelForVision2Seq.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    _attn_implementation="eager",
).to(DEVICE)

adapter_path = "allko14/SmolVLM-256M-Instruct-stf"
model_hf.load_adapter(adapter_path)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


adapter_config.json:   0%|          | 0.00/937 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

Testing a loaded pre-trained model on a test dataset

In [ ]:
file_name = 'hf_model_test_ds_batch_model_predictions.csv'
true_answers = list(map(extract_numbers, test_ds['solution']))
hf_model_predictions = []
with open(file_name, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Fine-tuning HF Model Prediction", "Correct Answer"])

    for i in tqdm(range(0, len(test_ds), 2)):
        examples = test_ds.select(range(i, i+2))
        batch = collate_fn(examples)
        inputs = batch.to(DEVICE)

        generated_ids2 = model_hf.generate(**inputs)
        generated_texts2 = processor.batch_decode(
            generated_ids2,
            skip_special_tokens=True,
        )

        ftm_pred = [(extract_numbers(gts.split('\nAnswer: ')[0].split('\nAssistant: ')[1])) for gts in generated_texts2]
        hf_model_predictions.extend(ftm_pred)
        for ft, ans in zip(ftm_pred, true_answers[i:i+2]):
            writer.writerow([ft, ans])


100%|██████████| 499/499 [11:01<00:00,  1.33s/it]


In [ ]:
files.download(file_name)
print(f'File {file_name} has been downlowded')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File hf_model_test_ds_batch_model_predictions.csv has been downlowded


In [ ]:
# calculating accuracy
accuracy_hf_batch = accuracy_score(hf_model_predictions, true_answers)
accuracy_hf_batch

1.0